# Import Library

In [ ]:
import os
import cv2
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow import keras
from tensorflow.keras.applications import MobileNet,EfficientNetB0, InceptionV3,ResNet50
from tensorflow.keras.layers import GlobalMaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score,recall_score, f1_score


# Load Data

In [ ]:
train_val_path = '/kaggle/input/skin-disease-dataset/skin-disease-datasaet/train_set'
test_path = '/kaggle/input/skin-disease-dataset/skin-disease-datasaet/test_set'

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='reflect'
)

train_data = []
val_data = []

train_class_counts = {}
val_class_counts ={}

for folder in sorted(os.listdir(train_val_path)):
    folder_path = os.path.join(train_val_path, folder)
    file = os.listdir(folder_path)
    num_train = int(0.8 * len(file))
    files_train = random.sample(file, num_train)
    files_val = list(set(file) - set(files_train))
    
    train_class_counts[folder] = 0
    val_class_counts[folder] = 0
    
    for file in files_train:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224)) 
        train_data.append((img, folder))

        img_array = np.expand_dims(img, axis=0)

        for _ in range(2):
            augmented_img = datagen.flow(img_array).next()[0].astype(np.uint8)
            train_data.append((augmented_img, folder))
        
        train_class_counts[folder] += 3  

    for file in files_val:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224))
        val_data.append((img, folder))
        
        val_class_counts[folder] += 1 

test_data = []
test_class_counts = {}

for folder in sorted(os.listdir(test_path)):
    folder_path = os.path.join(test_path, folder)
    files_test = os.listdir(folder_path)
    test_class_counts[folder] = 0
    for file in files_test:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224)) 
        test_data.append((img, folder))
        img_array = np.expand_dims(img, axis=0)
        test_class_counts[folder] += 1  
    
for class_name, count in train_class_counts.items():
    print(f"Kelas '{class_name}' dalam data TRAIN {count} gambar.")

for class_name, count in val_class_counts.items():
    print(f"Kelas '{class_name}' dalam data VALIDASI {count} gambar.")
    
for class_name, count in test_class_counts.items():
    print(f"Kelas '{class_name}' dalam data TEST {count} gambar.")


In [ ]:
import matplotlib.pyplot as plt

class_images = {}

for img, label in train_data:
    if label not in class_images:
        class_images[label] = img

fig, axes = plt.subplots(2, 4, figsize=(10, 5))
plt.suptitle('LABELS OF EACH IMAGE')

for (label, img), ax in zip(class_images.items(), axes.flatten()):
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.grid(True)
    ax.set_title(label)
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

plt.show()


# Perancangan Model 

In [ ]:
tf.keras.backend.clear_session()

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

num_classes = 8
x = GlobalMaxPooling2D()(base_model.output)

x = Dense(512, activation='relu', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
   
predictions = Dense(num_classes, activation='softmax')(x)


In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X_train, y_train = zip(*train_data)
X_val, y_val = zip(*val_data)

X_train = preprocess_input(np.array(X_train))
X_val = preprocess_input(np.array(X_val))

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)



# Training Model

In [ ]:
EPOCHS = 100
BATCH_SIZE = 128

best_model_dir = '/kaggle/working/'
early_stopping = EarlyStopping(monitor='val_loss', patience=15)
model_checkpoint = ModelCheckpoint(
    os.path.join(best_model_dir, 'best_model_EfficientNetB0.h5'),
    monitor='val_loss', 
    save_best_only=True,
    mode='min', 
    verbose=1
)

history = model.fit(X_train, y_train_one_hot,
                    validation_data=(X_val, y_val_one_hot),
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE,
                    callbacks=[early_stopping, model_checkpoint])


In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss,label='Training loss', marker='o')
plt.plot(epochs, val_loss,label='Validation loss', marker='o')
plt.title('Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
train_loss = history.history['accuracy']
val_loss = history.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss,label='Training accuracy', marker='o')
plt.plot(epochs, val_loss,label='Validation accuracy', marker='o')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()

plt.show()

# Evaluasi Model

In [ ]:
test_path = '/kaggle/input/skin-disease-dataset/skin-disease-datasaet/test_set'

In [ ]:
model = load_model('/kaggle/working/best_model_EfficientNetB0.h5') 

In [ ]:
real_label = []
predicted_class = []

for folder in sorted(os.listdir(test_path)):
    folder_path = os.path.join(test_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        img = preprocess_input(np.array([img]))  

        predictions = model.predict(img)
        real_label.append(folder)
        predicted_class_index = np.argmax(predictions)
        predicted_class.append(le.classes_[predicted_class_index])


In [ ]:
real_labels = np.array(real_label)
predicted_classes = np.array(predicted_class)

accuracy = accuracy_score(real_labels, predicted_classes)
precision = precision_score(real_labels, predicted_classes, average='weighted')
recall = recall_score(real_labels, predicted_classes, average='weighted')
f1 = f1_score(real_labels, predicted_classes, average='weighted')

print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
real_labels = np.array(real_label)
predicted_classes = np.array(predicted_class)

conf_matrix = confusion_matrix(real_labels, predicted_classes)

class_names = [str(i) for i in range(len(conf_matrix))] 
print(classification_report(real_labels, predicted_classes, target_names=class_names))


In [ ]:
conf_matrix = confusion_matrix(real_label, predicted_class)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Convert Model ke TFJS

In [ ]:
!tensorflowjs_converter --input_format keras best_model_EfficientNetB0.h5 tfjs/model